In [1]:
import mlflow

# Step 2: Set up the MLflow tracking server
# mlflow.set_tracking_uri("http://ec2-3-81-236-116.compute-1.amazonaws.com:5000/")

# Step 2: Set up the MLflow tracking server - local
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df = pd.read_csv("dataset.csv").dropna()
print(df.shape)

#  Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df["category"] = df["category"].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=["category"])

ngram_range = (1, 3)  # Trigram setting
max_features = 1000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_comment"], df["category"], test_size=0.2, random_state=42, stratify=df["category"]
)

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_vec, y_train)

c:\Data-Science\CampusX\Projects\youtube-comment-analyzer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(36662, 2)


In [2]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", f"{model_name}_triADASYSN")

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for LightGBM
def objective_xgboost(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)

    model = LGBMClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42,
    )
    return accuracy_score(y_test, model.fit(X_train_resampled, y_train_resampled).predict(X_test_vec))


# Step 7: Run Optuna for LightGBM classifier, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LGBMClassifier(
        n_estimators=best_params["n_estimators"],
        learning_rate=best_params["learning_rate"],
        max_depth=best_params["max_depth"],
        random_state=42,
    )

    # Log the best model with MLflow, passing the algo_name as "LightGBM"
    log_mlflow("LightGBM", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)


# Run the experiment for LightGBM
run_optuna_experiment()

[I 2024-12-02 11:14:19,190] A new study created in memory with name: no-name-144c9c21-7693-4144-90b8-75a3bdd9c86f


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.113136 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 100207
[LightGBM] [Info] Number of data points in the train set: 37441, number of used features: 975
[LightGBM] [Info] Start training from score -1.094720
[LightGBM] [Info] Start training from score -1.087801
[LightGBM] [Info] Start training from score -1.113492
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

[I 2024-12-02 11:14:22,249] Trial 0 finished with value: 0.5480703668348561 and parameters: {'n_estimators': 74, 'learning_rate': 0.0025920700171938173, 'max_depth': 3}. Best is trial 0 with value: 0.5480703668348561.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-ch

[I 2024-12-02 11:14:41,958] Trial 1 finished with value: 0.777035319787263 and parameters: {'n_estimators': 174, 'learning_rate': 0.06059419703212122, 'max_depth': 10}. Best is trial 1 with value: 0.777035319787263.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.080574 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 100207
[LightGBM] [Info] Number of data points in the train set: 37441, number of used features: 975
[LightGBM] [Info] Start training from score -1.094720
[LightGBM] [Info] Start training from score -1.087801
[LightGBM] [Info] Start training from score -1.113492


[I 2024-12-02 11:21:50,178] Trial 29 finished with value: 0.6828037638074458 and parameters: {'n_estimators': 54, 'learning_rate': 0.02918739429400857, 'max_depth': 10}. Best is trial 12 with value: 0.7913541524614756.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.078605 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 100207
[LightGBM] [Info] Number of data points in the train set: 37441, number of used features: 975
[LightGBM] [Info] Start training from score -1.094720
[LightGBM] [Info] Start training from score -1.087801
[LightGBM] [Info] Start training from score -1.113492
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

2024/12/02 11:22:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_ADASYN_TFIDF_Trigrams at: http://127.0.0.1:5000/#/experiments/602507174562575349/runs/0d6420c4ee0646769982e851f6d47900
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/602507174562575349
